Lets go through the steps that the robot takes during play.

Turn order:
(1) Determine the order of play - the robot doesn't need to know this until we start coding some AI decision making.

(2) Auction power plants
    Robot needs to know
     - what power plants are available
     - what power plants it has
     - what power plants other players have chosen for bidding
     - how many cities it has
    Robot will decide
     - if it needs to buy a new power plant or not
     - which power plant it will choose for bidding on
     - if it will join in the public bidding process
     - what the maximum it will pay for the power plant is
     - which power plant to discard if necessary
    Robot will update
     - what power plants it has
        - how many spaces it has for resources
     - how its resources are distributed
        - do any resources need throwing away
     - how much money it has

(3) Buying resources
    Robot needs to know
     - what space it has for resources
     - what resources it has
     - the cost of available resources
    Robot will decide
     - what resources it will buy
    Robot will update
     - what resources it has
     - where those resources are stored
     - how much money it has

(4) Building
    Robot needs to know
     - what the map is
        - what cities there are
        - what the connection costs are
     - what step we are in (step 1, 2 or 3)
     - what cities it has built on
     - what cities other players have built on
        - how many people have built there
        - what spaces are free, and their cost
     Robot will decide
      - if it will build a city
      - where to build the city
      - if it has enough money to build another city
     Robot will update
      - how much money it has
      - how many cities it has
        - where those cities are

(5) Bureaucracy
    Robot needs to know
     - how many cities it has built on
     - how many cities its power plants can supply
     - what resources it has
    Robot will decide
     - which resources to use to power the most cities
    The robot will then update
     - how much money it has
     - what resources it has
    


Variables and databases I need:
- a representation of the map
    - the cities on the map
    - the connections between the cities
    - the regions on the map
- the resource market
- the power plants
- a list of money earned depending on cities powered

- money
- each type of resource
- a list of power plants owned
- a list of cities in the robot's network

In [2]:
# What libraries / functions do we need to import? Put them in here.

import pyodbc
from pyodbc import Error

from databaseConnection import create_db_connection, read_query



In [1]:
# Set the starting variables

robotMoney = 50
mapName = "Japan"
powerPlantDeck = "standard"
step = 1

# Database access details:
user_name = input("Please give the user name for the database access.")
user_password = input(f"Please give the password for {user_name}")


# We will also need to set the robot options, but we'll code those later on

In [3]:
# Actually, we should initialise the instance of the Robot, so that it has money for the auction phase.

from robotClass import Robot

robot = Robot(robotMoney)

Robot.setRobotName(robot)

print(robot)


What should we call this robot?Bob
Bob has: Money = 50 and these powerplants: []


In [4]:
# Phase 2: Power plant auction - create the power plant market

from powerPlants import PowerPlant, createMarket

# We ask the user to tell us the costs of the current power plants in the market:
listOfPowerPlants = createMarket(step)

Market size is 8, because it is step 1.
What is the cost of the next power plant in the market?1
What is the cost of the next power plant in the market?2
What is the cost of the next power plant in the market?3
What is the cost of the next power plant in the market?6
What is the cost of the next power plant in the market?8
What is the cost of the next power plant in the market?11
What is the cost of the next power plant in the market?16
What is the cost of the next power plant in the market?33


In [5]:

power_plants = PowerPlant.updatePowerPlantMarket(user_name, user_password, listOfPowerPlants)

# We print some details to see what we have got.
print(listOfPowerPlants)
print(power_plants[1])


MySQL Database connection successful
[1, 2, 3, 6, 8, 11, 16, 33]
Cost: 2, Type: Oil, Resource: oil, Resource2: None, PowerCost: 2, CitiesPowered: 1


In [14]:
# Now we will bid on some power plants
# We will start with the robot rule 'second smallest number'

# If I have a mix of 'print' and 'input' messages, the printed messages appear below this block, while the input
# messages appear in a box. This is confusing and hides some of the information I am trying to communicate.
# BUT when I view the Jypter notebook as a webpage, that problem goes away....

# The robot will ask if it is their turn.
myTurn = input("Is it my turn to choose a power plant to bid on? Y or N?")

didIbuyYet = "n"

# If it is step 1 or 2, we need to split the existing market into the current and future market.
# If it is step 3, then all 6 power plants are in the current market
# Only the current market is of interest to the robot.

def createCurrentMarket(listOfPowerPlants):    
    if step == 3:
        currentMarket = listOfPowerPlants
    else:
        listOfPowerPlants.sort()
        currentMarket = listOfPowerPlants[:-4]
        
    return currentMarket

currentMarket = createCurrentMarket(listOfPowerPlants)
print(f"The current market is {currentMarket}")

# I should put in a comment as to why I am doing this here.
# What am I doing here????
# I don't think I need this - but I will just comment it out.
# currentMarketDetails = PowerPlant.updatePowerPlantMarket(user_name, user_password, currentMarket)

# If it is not the robot's turn to choose, then for this robot we just need to update
# the market. For other versions, we will need to check what power plant is being bid on
# and what the bids are - but that is for later on.

while didIbuyYet == 'n':

    while myTurn.lower() == "n":
        boughtPowerPlant = input("I won't bid, so just tell me which power plant was bought. ")
        listOfPowerPlants.remove(int(boughtPowerPlant))
        # This code is repeated in the block below, so we could hive it off as a function
        # when we refactor.
        newPowerPlant = int(input("What is the cost of the new power plant you have drawn? "))
        listOfPowerPlants.append(newPowerPlant)
        currentMarket = createCurrentMarket(listOfPowerPlants)
        print(f"Power plant {newPowerPlant} was drawn and the current market is {currentMarket}")
        myTurn = input("Is it now my turn to choose a power plant to bid on? Y or N? ")
    
    # If it is the robot's turn to choose, then they will keep on choosing until they buy something.
    # So we only care if 'myTurn' is 'n' - otherwise we just run this block.
    choice = currentMarket[1]
    print(f"I bid {choice} electros for power plant {choice}.")
    otherBid = input(f"Does anyone else want to bid on power plant {choice}? Y or N? ")
    if otherBid.lower() == "y":
        print(f"Ok, I pass on {choice}.")
        listOfPowerPlants.remove(choice)
        # The next 4 lines are a repeat of lines above, so we can hive it off as a function later.
        newPowerPlant = int(input("What is the cost of the new power plant you have drawn? "))
        listOfPowerPlants.append(newPowerPlant)
        currentMarket = createCurrentMarket(listOfPowerPlants)
        print(f"Power plant {newPowerPlant} was drawn and the current market is {currentMarket}")
    elif otherBid.lower() == "n":
        didIbuyYet = 'y'
        print(f"Ok, I will buy power plant {choice} for {choice} electros")

    


Robot.deductRobotMoney(robot, choice)
print(f"I have spent {choice} electros, and now I have {robot.money} electros.")
Robot.addPowerplant(robot, choice)
print(f"I now have these power plants: {robot.powerPlants}")
    
    



Is it my turn to choose a power plant to bid on? Y or N?n
The current market is [6, 8, 11, 28]
I won't bid, so just tell me which power plant was bought. 6
What is the cost of the new power plant you have drawn? 40
Power plant 40 was drawn and the current market is [8, 11, 28, 28]
Is it now my turn to choose a power plant to bid on? Y or N? y
I bid 11 electros for power plant 11.
Does anyone else want to bid on power plant 11? Y or N? n
Ok, I will buy power plant 11 for 11 electros
I have spent 11 electros, and now I have 10 electros.
I now have these power plants: [6, 8, 11]


In [ ]:
# Now that the robot has a power plant, it needs to decide what resources to buy.
# For now it will just buy basic resources to power the power plant for 1 turn.
# It needs to know:
# What is the price point of all the resources
# What resources it already has
# 